In [1]:
import pandas as pd
import nltk 
from nltk import tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.model_selection import cross_val_predict, cross_val_score
import language_tool_python

stop_words = set(stopwords.words('portuguese'))
stop_words = list(stop_words )


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_excel('Classificação Artigos.xlsx', sheet_name = 'articles')
df.head(3)

,title,text,date,category,Classificação,link,ano,mês,dia
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,Corrupção,http://www1.folha.uol.com.br/poder/2017/10/192...,2017,9,10
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...,2017,9,10
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017,9,10


In [3]:
substituicoes = {
    'ilustrada': 'ilustrada/ilustrissima',
    'ilustrissima': 'ilustrada/ilustrissima'
    }

# Substituir usando o método replace com o dicionário de mapeamento
df['category'].replace(substituicoes, inplace=True)

In [4]:
qtd = df['category'].value_counts()
qtd = qtd[qtd >= 400].index.tolist()
qtd

['poder',
 'colunas',
 'mercado',
 'ilustrada/ilustrissima',
 'mundo',
 'cotidiano',
 'esporte',
 'opiniao',
 'paineldoleitor',
 'saopaulo',
 'tec',
 'tv',
 'educacao',
 'turismo',
 'ciencia',
 'equilibrioesaude',
 'folhinha',
 'empreendedorsocial',
 'comida',
 'ambiente']

In [5]:
df['category'].value_counts()

category
poder                           22022
colunas                         21622
mercado                         20970
ilustrada/ilustrissima          17756
mundo                           17130
cotidiano                       16967
esporte                         16871
opiniao                          4525
paineldoleitor                   4011
saopaulo                         3955
tec                              2260
tv                               2142
educacao                         2118
turismo                          1903
ciencia                          1335
equilibrioesaude                 1312
folhinha                          876
empreendedorsocial                841
comida                            828
ambiente                          491
o-melhor-de-sao-paulo             189
guia-de-livros-discos-filmes      143
especial                           43
guia-de-livros-filmes-discos       28
multimidia                         27
mulher                             16
eur

In [6]:
df = df[df['category'].isin(qtd)]
df.head(3)

,title,text,date,category,Classificação,link,ano,mês,dia
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,Corrupção,http://www1.folha.uol.com.br/poder/2017/10/192...,2017,9,10
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada/ilustrissima,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...,2017,9,10
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...,2017,9,10


In [7]:
df = df[~df['category'].isin(['colunas', 'cotidiano', 'opiniao', 'saopaulo', 'paineldoleitor', 'tv'])]
df['category'].value_counts()

category
poder                     22022
mercado                   20970
ilustrada/ilustrissima    17756
mundo                     17130
esporte                   16871
tec                        2260
educacao                   2118
turismo                    1903
ciencia                    1335
equilibrioesaude           1312
folhinha                    876
empreendedorsocial          841
comida                      828
ambiente                    491
Name: count, dtype: int64

In [8]:
df_sample = pd.DataFrame()

for category in df['category'].unique():
    sample_rows = df[df['category'] == category].sample(n=400, random_state=42)
    df_sample = pd.concat([df_sample, sample_rows])

df_sample.reset_index(drop = True, inplace = True)

df_sample.head(3)

,title,text,date,category,Classificação,link,ano,mês,dia
0,"Com receio de desgaste, Lula diz que não briga...",Em jantar com a presidente Dilma Rousseff nest...,2016-03-22,poder,NaN,http://www1.folha.uol.com.br/poder/2016/03/175...,2016,3,22
1,Queiroz Galvão tirou do país diretor que pagav...,"Representante do ""alto escalão"" da Queiroz Gal...",2016-02-08,poder,NaN,http://www1.folha.uol.com.br/poder/2016/08/179...,2016,2,8
2,Cabral e Kassab lideram ranking de repasses il...,O ex-governador Sérgio Cabral (PMDB) e outros ...,2017-04-14,poder,NaN,http://www1.folha.uol.com.br/poder/2017/04/187...,2017,4,14


In [9]:
df_sample['category'].value_counts()

category
poder                     400
ilustrada/ilustrissima    400
mercado                   400
mundo                     400
esporte                   400
tec                       400
ambiente                  400
equilibrioesaude          400
educacao                  400
ciencia                   400
turismo                   400
empreendedorsocial        400
comida                    400
folhinha                  400
Name: count, dtype: int64

In [10]:
df_sample['text'] = df_sample['text'].str.lower().str.replace('/n', ' ')

df_sample['text'] = df_sample['text'].apply(lambda x: re.sub(r'^\s*|\s*$|[^\w\s]|[\d]', ' ', str(x)))
df_sample.head(3)

,title,text,date,category,Classificação,link,ano,mês,dia
0,"Com receio de desgaste, Lula diz que não briga...",em jantar com a presidente dilma rousseff nes...,2016-03-22,poder,NaN,http://www1.folha.uol.com.br/poder/2016/03/175...,2016,3,22
1,Queiroz Galvão tirou do país diretor que pagav...,representante do alto escalão da queiroz ga...,2016-02-08,poder,NaN,http://www1.folha.uol.com.br/poder/2016/08/179...,2016,2,8
2,Cabral e Kassab lideram ranking de repasses il...,o ex governador sérgio cabral pmdb e outros...,2017-04-14,poder,NaN,http://www1.folha.uol.com.br/poder/2017/04/187...,2017,4,14


In [11]:
df_sample['title'] = df_sample['title'].str.lower().str.replace('/n', ' ')

df_sample['title'] = df_sample['title'].apply(lambda x: re.sub(r'^\s*|\s*$|[^\w\s]|[\d]', ' ', str(x)))
df_sample.head(3)

,title,text,date,category,Classificação,link,ano,mês,dia
0,com receio de desgaste lula diz que não brig...,em jantar com a presidente dilma rousseff nes...,2016-03-22,poder,NaN,http://www1.folha.uol.com.br/poder/2016/03/175...,2016,3,22
1,queiroz galvão tirou do país diretor que paga...,representante do alto escalão da queiroz ga...,2016-02-08,poder,NaN,http://www1.folha.uol.com.br/poder/2016/08/179...,2016,2,8
2,cabral e kassab lideram ranking de repasses i...,o ex governador sérgio cabral pmdb e outros...,2017-04-14,poder,NaN,http://www1.folha.uol.com.br/poder/2017/04/187...,2017,4,14


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df_sample['text'], df_sample['category'], 
                                                    test_size=0.30, 
                                                    random_state=32)

In [21]:
# MultinomialNB
classify = Pipeline(
                [('vect', CountVectorizer(ngram_range=(1,2),stop_words= stop_words)),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB(fit_prior=False, alpha=.01)),
                 ])

In [22]:
classify.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(ngram_range=(1, 2),
                                 stop_words=['da', 'dele', 'aquela', 'estivera',
                                             'houverem', 'nossa', 'houveremos',
                                             'teria', 'esteve', 'hajamos',
                                             'estes', 'houveram', 'esta',
                                             'tivermos', 'sem', 'fôssemos',
                                             'muito', 'por', 'fomos', 'quando',
                                             'tivemos', 'delas', 'houver',
                                             'também', 'nosso', 'depois',
                                             'esses', 'hão', 'ser', 'seu', ...])),
                ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB(alpha=0.01, fit_prior=False))])

In [23]:
# validação cruzada - separa em N (cv) partes para treinar e classificar

scores = cross_val_score(classify, X_test, y_test, cv=5, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.83 (+/- 0.03)


In [24]:
preds = classify.predict(X_test)
print(metrics.classification_report(y_test, preds))

                        precision    recall  f1-score   support

              ambiente       0.79      0.91      0.85       114
               ciencia       0.80      0.62      0.69       120
                comida       0.93      0.92      0.93       119
              educacao       0.89      0.94      0.91       113
    empreendedorsocial       0.84      0.90      0.87       122
      equilibrioesaude       0.80      0.90      0.84       135
               esporte       0.99      0.92      0.96       140
              folhinha       0.95      0.78      0.86       120
ilustrada/ilustrissima       0.85      0.84      0.84       117
               mercado       0.80      0.73      0.76       113
                 mundo       0.82      0.87      0.85       118
                 poder       0.86      0.90      0.88       119
                   tec       0.78      0.82      0.80       114
               turismo       0.86      0.88      0.87       116

              accuracy                